In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Extract

In [2]:
!mkdir dataset

In [3]:
import zipfile
zipfile.ZipFile('/content/drive/MyDrive/yooninahong/dataset.zip').extractall('/content/dataset')

## Download yolov5

In [4]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10537, done.
remote: Total 10537 (delta 0), reused 0 (delta 0), pack-reused 10537
Receiving objects: 100% (10537/10537), 10.68 MiB | 31.35 MiB/s, done.
Resolving deltas: 100% (7287/7287), done.


In [5]:
!pip install -r /content/yolov5/requirements.txt

     |████████████████████████████████| 596 kB 5.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## make img list

In [6]:
from glob import glob

train_img_list = glob('/content/dataset/images/train/*.jpg')
val_img_list = glob('/content/dataset/images/val/*.jpg')

In [7]:
val_img_list[:3]

['/content/dataset/images/val/block75.jpg',
 '/content/dataset/images/val/brailleblock46.jpg',
 '/content/dataset/images/val/bike_152.jpg']

### Make yaml file

In [8]:
with open('/content/dataset/train.txt','w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/dataset/val.txt','w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [9]:
import yaml
import os
data = dict()
data['train'] = '/content/dataset/train.txt'
data['val'] = '/content/dataset/val.txt'
data['nc'] = '5'
data['names'] = ['electricscooter','brailleblock','person','bike','motorcycle_F']

with open('/content/dataset/data.yaml','w') as f:
  yaml.dump(data,f)
print(data)

{'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt', 'nc': '5', 'names': ['electricscooter', 'brailleblock', 'person', 'bike', 'motorcycle_F']}


In [10]:
with open('/content/dataset/data.yaml', 'r') as f:
  data = yaml.load(f,Loader = yaml.Loader)
print(data)

{'names': ['electricscooter', 'brailleblock', 'person', 'bike', 'motorcycle_F'], 'nc': '5', 'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt'}


### Train YOLOv5

In [11]:
!python /content/yolov5/train.py --img 640 --batch 64 --epoch 100 --data /content/dataset/data.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name ynh_result

train: weights=yolov5s.pt, cfg=/content/yolov5/models/yolov5s.yaml, data=/content/dataset/data.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=100, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=ynh_result, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-190-ge7bf382 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_g

In [22]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/ynh_result/weights/best.pt --img 640 --conf 0.5 --source /content/KakaoTalk_20210811_203456004_09.jpg


detect: weights=['/content/yolov5/runs/train/ynh_result/weights/best.pt'], source=/content/KakaoTalk_20210811_203456004_09.jpg, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-190-ge7bf382 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/KakaoTalk_20210811_203456004_09.jpg: 640x480 1 electricscooter, Done. (0.015s)
Speed: 0.5ms pre-process, 14.8ms inference, 1.7ms NMS per image at shape (1, 3, 640, 640)
Results saved to yolov5/runs/detect/exp10
